In [57]:
import pandas as pd

from data.points import ANCHORS

df = pd.read_csv("/home/piotr/Documents/mgr/uwb-rs/data-processing/data/2023-05-21-20:44:03.341245P3-1-h.csv", index_col=0)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df[df.anchor.isin(ANCHORS.keys())]

In [58]:
factor = 3

data = dict()
data["timestamp"] = df[df.anchor == df.anchor.iloc[0]]["timestamp"].to_list()

for anchor in ANCHORS.keys():
    data[anchor] = df[df.anchor == anchor]["distance"].rolling(window=factor).mean().to_list()
df = pd.DataFrame(data, index=data["timestamp"])
df.pop("timestamp")
df.dropna(inplace=True)


In [59]:
from easy_trilateration.model import *
from easy_trilateration.least_squares import easy_least_squares, solve_history
from easy_trilateration.graph import *
from helpers import distance

history = []

for timestamp, row in df.iterrows():
    c = [Circle(*ANCHORS[anchor], row[anchor]) for anchor in ANCHORS.keys()]
    history.append(Trilateration(c))

solve_history(history)


In [60]:
# import matplotlib.pyplot as plt
# from matplotlib.lines import Line2D

# ax = plt.subplot()
# ax.add_line(Line2D((0,0),(1,1)))
# a = animate(history, ax)
# # with open("video.html", "w") as file:
# #     file.write(a.to_html5_video())
# # pass

In [61]:
data = {
    "x": [],
    "y": [],
    "radius": [],
    "timestamp": df.index.to_list()
}
for t in history:
    t: Trilateration
    data["x"].append(t.result.center.x)
    data["y"].append(t.result.center.y)
    data["radius"].append(t.result.radius)

df = pd.DataFrame(data)

In [63]:
import plotly.express as px

px.scatter(df, x="x", y="y", animation_frame="timestamp",
        #    size="radius",
           range_x=[0,2000], range_y=[0,4000],
           width=600, height=600)